In [42]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Import

In [43]:
from __future__ import print_function
import numpy as np
import pandas as pd
from math import sqrt, pow


from ipywidgets import interact, interactive, fixed, interact_manual, FileUpload
import ipywidgets as widgets
from IPython.display import display
import io

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# User Input

After uploading the sensor data, the user defines survey scenario at survey depth.

In [44]:
upload = FileUpload(accept= '.xlsx', multiple = False)
display(upload)

FileUpload(value={}, accept='.xlsx', description='Upload')

In [45]:
with open('Sensors_DataExcel.xlsx', 'wb') as output_file:
    for uploaded_filename in upload.value:
        content = upload.value[uploaded_filename]['content']
        output_file.write(content)

27115

In [46]:
filename = 'Sensors.xlsx'
head = 4
xls = pd.ExcelFile(content)

In [47]:
# widget for input of desired survey depth
depth_range_slider = widgets.IntRangeSlider(
    value=[100,2000],
    min=0.2,
    max=11000,
    step=10,
    description='Survey Depths:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout={'width': '990px'}
)
display(depth_range_slider)

IntRangeSlider(value=(100, 2000), continuous_update=False, description='Survey Depths:', layout=Layout(width='…

In [48]:
survey_depth = [depth_range_slider.value[0],depth_range_slider.value[1]]

In [49]:
# selects tables. Keywords are: nautical, seabed, oceanic, geophysical (bathymetric survey, seabed imaging, oceanographic survey, geophysical survey)
#survey_scenario = ['nautical', 'seabed', 'oceanic', 'geophysical']

# widget for input of survey scenarios
scenario_select = widgets.SelectMultiple(
    options=['Bathymetric Survey', 'Seafloor Classification', 'Oceanographic Survey', 'Geophysical Survey'],
    value=[],
    rows=5,
    description='Purpose',
    disabled=False
)

display(scenario_select)

SelectMultiple(description='Purpose', options=('Bathymetric Survey', 'Seafloor Classification', 'Oceanographic…

In [51]:
# getting chosen scenario values
selected = scenario_select.value  
survey_scenario = []

for i in selected:
    survey_scenario.append(i)


In [52]:
# since the selection is highly dependent on depth, the approximate transducer draught has to be considered. The min depth in the datasheets 
# refers to depth below transducer. This means the minimum water depth that can be measured is: given number + ship draught + mounting depth
# approximate ship draught
ship_draught = 3.71

In [53]:
# for the second step: maximum space available in mm, kg
keel_width = 1200
keel_length = 9000
weight_limit = 10000
space_limits = [keel_width, keel_length, weight_limit]

# Step 1: Functions for Selection by Survey Scenario

## MBES: mbes_scenario(depth), return df


In [54]:
def mbes_scenario(depth):
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'MBES':
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    # The sensor should be able to measure 10 percent more than maximum survey depth 
    bedingung1 = ((df.max_depth-df.max_depth*0.1) >= depth[1]) 
    # draught is considered for minimum survey depth = transducer draught + 10% Spielraum. 
    bedingung2 = (df.min_depth+ship_draught) <= depth[0]  
    
    # apply to df
    df = df[bedingung1 & bedingung2]

    return df

## SBES: sbes_scenario(depth), return df

In [55]:
def sbes_scenario(depth):
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBES': 
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    # The sensor should be able to measure 10 percent more than maximum survey depth 
    bedingung1 = ((df.max_depth-df.max_depth*0.1) >= depth[1]) 
    # draught is considered for minimum survey depth = transducer draught + 10% Spielraum. 
    bedingung2 = (df.min_depth+ship_draught) <= depth[0]  
    
    # apply to df
    df = df[bedingung1 & bedingung2]
    
    return df

## SBP: sbp_scenario(depth), return df

In [56]:
def sbp_scenario(depth):
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBP': 
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    # The sensor should be able to measure 10 percent more than maximum survey depth 
    bedingung1 = ((df.max_depth-df.max_depth*0.1) >= depth[1]) 
    # Um klar vom nearfield zu sein, werden Tiefgang und mounting berücksichtigt = transducer draught + 10% Spielraum.
    bedingung2 = (df.min_depth+ship_draught) <= depth[0]  
    
    # apply to df
    df = df[bedingung1 & bedingung2]

    return df

## SSS: sss_scenario(depth), return df

In [57]:
def sss_scenario(depth):
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SSS': 
            df= xls.parse(sheet_name, skiprows = head)
    
    #NaN values are replaced by 0
    df = df.replace(np.nan, 0)
    
    tow_height = 0.15*df.swath_width
    # depth_rating
    bedingung3 = (depth[1]-tow_height) < df.depth_rating
    
    # Maske mit Bedingungen auf df anwenden
    df = df[bedingung3]
    #print('bedingung1 erfüllt: ',bedingung1)
    #print('bedingung2 erfüllt: ', bedingung2)
    
    return df

# Call Functions for Selection by Survey Scenario

In [58]:
depth = [survey_depth]

for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        #print('Instruments for nautical survey: ')
        for i in depth:
            nautical_mbes = mbes_scenario(i)
            nautical_sbes = sbes_scenario(i)
            nautical_sbp = sbp_scenario(i)
            
    if 'Seafloor Classification' in i:
        #print('Instruments for seabed investigation: ')
        for i in depth:
            seabed_mbes = mbes_scenario(i)
            seabed_sbp = sbp_scenario(i)
            seabed_sss = sss_scenario(i)
            
    if 'Oceanographic Survey' in i:
        #print('Instruments for investigation of the water column: ')
        for i in depth:
            oceanic_mbes = mbes_scenario(i) # hier noch Auswahlmöglichkeit einbauen, dass nur scientific ausgewählt werden
            #print('other not yet defined')
            
    if 'Geophysical Survey' in i:
        #print('Instruments for underground features: ')
        for i in depth:
            print('')
            

In [59]:
# prepare lists for next step

mbes_sensors = []
sbes_sensors = []
sbp_sensors = []
sss_sensors = []

for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        mbes_sensors.append(nautical_mbes)
        sbes_sensors.append(nautical_sbes)
        sbp_sensors.append(nautical_sbp)
        #sss_sensors.append(nautical_sss)
    if 'Seabed Classification' in i:
        mbes_sensors.append(seabed_mbes)
        #sbes_sensors.append(seabed_sbes)
        sbp_sensors.append(seabed_sbp)
        sss_sensors.append(seabed_sss)
    if 'Oceanographic Survey' in i:
        mbes_sensors.append(oceanic_mbes)
        #sbes_sensors.append(oceanic_sbes)
        #sbp_sensors.append(oceanic_sbp)
        #sss_sensors.append(oceanic_sss)
    if 'Geophysical Survey' in i:
        #mbes_sensors.append(geophysical_mbes)
        #sbes_sensors.append(geophysical_sbes)
        #sbp_sensors.append(geophysical_sbp)
        #sss_sensors.append(geophysical_sss)
        pass

In [60]:
# concat the double df for mbes/sbes/sbp/sss and drop duplicates, only for not empty lists

if not mbes_sensors:
    all_mbes = []
else:
    all_mbes = pd.concat(mbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbes_sensors:
    all_sbes = []
else:
    all_sbes = pd.concat(sbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbp_sensors:
    all_sbp = []
else:
    all_sbp = pd.concat(sbp_sensors).drop_duplicates().reset_index(drop=True)

if not sss_sensors:
    all_sss = []
else:
    all_sss = pd.concat(sss_sensors).drop_duplicates().reset_index(drop=True)

In [61]:
# OPTIONAL:
# cut all instruments that are overachieving (measure more than twice the required maximum depth)
all_mbes.drop(all_mbes[(all_mbes.max_depth)> (survey_depth[1])*2].index)
all_sbes.drop(all_sbes[(all_sbes.max_depth)> (survey_depth[1])*2].index)
all_sbp.drop(all_sbp[(all_sbp.max_depth)> (survey_depth[1])*2].index)

,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation,Unnamed: 38
4,EM712 0.5x0.5,Kongsberg,13.07.2021,bathymetry,3.0,3600.0,0.0,0.0,1940.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
5,EM712 0.5x1,Kongsberg,13.07.2021,bathymetry,3.0,3400.0,0.0,0.0,1940.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
6,EM712 1x1,Kongsberg,13.07.2021,bathymetry,3.0,3300.0,0.0,0.0,970.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
7,EM712 1x2,Kongsberg,13.07.2021,bathymetry,3.0,3300.0,0.0,0.0,970.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
8,EM712 2x2,Kongsberg,13.07.2021,bathymetry,3.0,3000.0,0.0,0.0,490.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
16,SeaBat7160,Teledyne RESON,29.09.2021,bathymetry,10.0,3000.0,0.0,0.0,1477.0,1100.0,...,0.0,yes,yes,0.0,0,0,no,yes,"roll, pitch",0


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation,Unnamed: 38
0,3300 4x4,Edgetech,29.09.2021,0.0,0.0,3000,0.0,80.0,0.0,0.0,...,0,0.0,0.0,no,no,yes,0.0,0,0,0
2,3300 dice 5,Edgetech,29.09.2021,0.0,0.0,3000,0.0,150.0,0.0,0.0,...,0,0.0,0.0,no,no,yes,0.0,0,0,0
10,EchoTracMKIII HM200,Teledyne Odom,29.09.2021,0.0,0.2,4000,0.0,0.0,0.0,0.0,...,3.5-50 kHz,0.0,0.0,yes,0,0,0.0,0,0,0
11,EchoTracMKIII HM200,Teledyne Odom,29.09.2021,0.0,0.2,4000,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0,0,0


In [62]:
print('The following instruments were selected for the given survey scenarios: ')
print('\n', 'MBES models')
all_mbes
print('\n', 'SBES models')
all_sbes
print('\n', 'SBP models')
all_sbp
print('\n', 'SSS models')
all_sss

The following instruments were selected for the given survey scenarios: 

 MBES models


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation,Unnamed: 38
0,EM304 0.5x0.5,Kongsberg,13.07.2021,bathymetry,10.0,8000.0,0.0,0.0,0.0,0.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
1,EM304 1x1,Kongsberg,13.07.2021,bathymetry,10.0,8000.0,0.0,0.0,0.0,0.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
2,EM304 2x2,Kongsberg,13.07.2021,bathymetry,10.0,8000.0,0.0,0.0,0.0,0.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
3,EM304 4x4,Kongsberg,13.07.2021,bathymetry,10.0,8000.0,0.0,0.0,0.0,0.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
4,EM712 0.5x0.5,Kongsberg,13.07.2021,bathymetry,3.0,3600.0,0.0,0.0,1940.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
5,EM712 0.5x1,Kongsberg,13.07.2021,bathymetry,3.0,3400.0,0.0,0.0,1940.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
6,EM712 1x1,Kongsberg,13.07.2021,bathymetry,3.0,3300.0,0.0,0.0,970.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
7,EM712 1x2,Kongsberg,13.07.2021,bathymetry,3.0,3300.0,0.0,0.0,970.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
8,EM712 2x2,Kongsberg,13.07.2021,bathymetry,3.0,3000.0,0.0,0.0,490.0,224.0,...,0.0,yes,yes,0.0,yes,yes,yes,yes,"roll, pitch, yaw",0
9,HydroSweepDS 0.5x1,Teledyne RESON,29.09.2021,bathymetry,10.0,11000.0,0.0,0.0,10373.0,299.0,...,0.0,yes,yes,0.0,no,yes,yes,yes,"roll, pitch, yaw",parametric sbp



 SBES models


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation
0,EA640,Kongsberg,2021-09-29,bathymetry,1.0,12500.0,0.0,0.0,213.0,438.0,...,12-500 kHz,0.0,0.0,0.0,0.0,yes,yes,0.0,0.0,0.0



 SBP models


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation,Unnamed: 38
0,3300 4x4,Edgetech,29.09.2021,0.0,0.0,3000,0.0,80.0,0.0,0.0,...,0,0.0,0.0,no,no,yes,0.0,0,0,0
1,3300 5x5,Edgetech,29.09.2021,0.0,0.0,5000,0.0,80.0,0.0,0.0,...,0,0.0,0.0,no,no,yes,0.0,0,0,0
2,3300 dice 5,Edgetech,29.09.2021,0.0,0.0,3000,0.0,150.0,0.0,0.0,...,0,0.0,0.0,no,no,yes,0.0,0,0,0
3,3300 hexagonal,Edgetech,29.09.2021,0.0,0.0,5000,0.0,150.0,0.0,0.0,...,0,0.0,0.0,no,no,yes,0.0,0,0,0
4,deep-36,Innomar,29.09.2021,0.0,5.0,6000,0.0,150.0,900.0,900.0,...,2-7 kHz,0.0,0.0,yes,no,yes,0.0,0,"heave, roll",0
5,Echoes3 3500 T7,iXblue,29.09.2021,0.0,1.0,6000,0.0,150.0,980.0,980.0,...,0,0.0,0.0,0,no,yes,0.0,0,0,0
6,SBP29 3,Kongsberg,29.09.2021,0.0,10.0,11000,0.0,200.0,7450.0,800.0,...,0,0.0,0.0,no,yes,yes,0.0,0,"roll, pitch, yaw",hydrophone shared with EM122/302
7,SBP29 6,Kongsberg,29.09.2021,0.0,10.0,11000,0.0,200.0,3834.0,800.0,...,0,0.0,0.0,no,yes,yes,0.0,0,"roll, pitch, yaw",hydrophone shared with EM122/302
8,SBP29 12,Kongsberg,29.09.2021,0.0,10.0,11000,0.0,200.0,1970.0,800.0,...,0,0.0,0.0,no,yes,yes,0.0,0,"roll, pitch, yaw",hydrophone shared with EM122/302
9,TOPAS PS18,Kongsberg,29.09.2021,0.0,20.0,11000,0.0,200.0,1142.0,1140.0,...,0.5-6.0 kHz,0.0,0.0,yes,yes,yes,0.0,0,0,SAS processing



 SSS models


[]

# Step 2: Functions for Selection by Space Restrictions

## MBES: mbes_space(space_limits), return df

In [63]:
# df from first step, space limits from user input
def mbes_space(space_limits, df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # remember: sensor without dimension values are set to 0. They will be selected!
    # the transmitting array is arranged along the keel, so its length plus the rx perpendicular width is the limit
    bedingung1 = (df.tx_length+df.rx_width) <= length 
    # the receiving array is arranged perpendicular to the transmitting array (mills cross config) so its width is the limit
    bedingung2 = (df.rx_length) <= width
    # the weight for the whole transducer setup (tx+rx) shall not exceed the specification
    #bedingung3 = (df.tx_weight+df.rx_weight) <= weight 

    # apply to df
    #df = df[bedingung1 & bedingung2 & bedingung3]
    df = df[bedingung1 & bedingung2]
    #print('bedingung1 erfüllt: ',bedingung1)
    #print('bedingung2 erfüllt: ', bedingung2)
    #print('bedingung3 erfüllt: ', bedingung3)
    return df

# SBES: sbes_space(space_limits), return df

In [64]:
def sbes_space(space_limits,df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBES':
            df= xls.parse(sheet_name, skiprows = head)
            
    bedingung1 = (df.tx_length) <= length
    bedingung2 = (df.tx_width) <= width
    bedingung3 = (df.tx_weight) <= weight
    
    # apply to df
    df = df[bedingung1 & bedingung2 & bedingung3]
    
    return df

## SBP: sbp_space(space_limits), return df

In [65]:
def sbp_space(space_limits,df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SBP':
            df= xls.parse(sheet_name, skiprows = head)
            
    bedingung1 = (df.tx_length) <= length
    bedingung2 = (df.tx_width) <= width
    bedingung3 = (df.tx_weight) <= weight
    
    # apply to df
    df = df[bedingung1 & bedingung2 & bedingung3]
    
    return df

## SSS: sss_space(space_limits), return df

Hier muss unterschieden werden nach Geräten, die fest montiert sind und Geräten, die geschleppt werden. Nur für feste Geräte muss Platzbedarf geprüft werden, für geschleppte ist Hinweis, dass das Gerät geschleppt wird, sinnvoll. 

In [66]:
def sss_space(space_limits,df):
    length = space_limits[1]
    width = space_limits[0]
    weight = space_limits[2]
    
    # parse applicable excel sheet
    for sheet_name in xls.sheet_names:
        if sheet_name == 'SSS': 
            df= xls.parse(sheet_name, skiprows = head)
            
    bedingung1 = (df.tx_length) <= length
    bedingung2 = (df.tx_width) <= width
    bedingung3 = (df.tx_weight) <= weight
    # nur fest installierte Geräte
    bedingung4 = (df.pole == 'no') & (df.towfish == 'no')
    df = df[bedingung1 & bedingung2 & bedingung3 & bedingung4]
    
    return df

# Call Functions for Selection by Space

In [67]:
for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        #print('Instruments for nautical survey: ')
        nautical_mbes = mbes_space(space_limits,nautical_mbes)
        nautical_sbes = sbes_space(space_limits, nautical_sbes)
        nautical_sbp = sbp_space(space_limits, nautical_sbp)
        
    if 'Seafloor Classification' in i:
        #print('Instruments for seabed investigation: ')
        seabed_mbes = mbes_space(space_limits, seabed_mbes)
        seabed_sbp = sbp_space(space_limits,seabed_sbp)
        seabed_sss = sss_space(space_limits, seabed_sss)
        
    if 'Oceanographic Survey' in i:
        #print('Instruments for investigation of the water column: '
        oceanic_mbes = mbes_space(space_limits,oceanic_mbes)
    
    if 'Geophysical Survey' in i:
        #print('Instruments for underground features: ')
        pass

In [68]:
# prepare lists for next step

mbes_sensors = []
sbes_sensors = []
sbp_sensors = []
sss_sensors = []

for i in survey_scenario:
    if 'Bathymetric Survey' in i:
        mbes_sensors.append(nautical_mbes)
        sbes_sensors.append(nautical_sbes)
        sbp_sensors.append(nautical_sbp)
        #sss_sensors.append(nautical_sss)
    if 'Seafloor Classification' in i:
        mbes_sensors.append(seabed_mbes)
        #sbes_sensors.append(seabed_sbes)
        sbp_sensors.append(seabed_sbp)
        sss_sensors.append(seabed_sss)
    if 'Oceanographic Survey' in i:
        mbes_sensors.append(oceanic_mbes)
        #sbes_sensors.append(oceanic_sbes)
        #sbp_sensors.append(oceanic_sbp)
        #sss_sensors.append(oceanic_sss)
    if 'Geophysical Survey' in i:
        #mbes_sensors.append(geophysical_mbes)
        #sbes_sensors.append(geophysical_sbes)
        #sbp_sensors.append(geophysical_sbp)
        #sss_sensors.append(geophysical_sss)
        pass

In [69]:
# concat the double df for mbes/sbes/sbp/sss and drop duplicates, only for not empty lists
if not mbes_sensors:
    all_mbes = []
else:
    all_mbes = pd.concat(mbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbes_sensors:
    all_sbes = []
else:
    all_sbes = pd.concat(sbes_sensors).drop_duplicates().reset_index(drop=True)
    
if not sbp_sensors:
    all_sbp = []
else:
    all_sbp = pd.concat(sbp_sensors).drop_duplicates().reset_index(drop=True)

if not sss_sensors:
    all_sss = []
else:
    all_sss = pd.concat(sss_sensors).drop_duplicates().reset_index(drop=True)


In [70]:
# concat the double df for mbes/sbes/sbp/sss and drop duplicates
all_mbes = pd.concat([nautical_mbes, seabed_mbes, oceanic_mbes]).drop_duplicates().reset_index(drop=True)
all_sbes = pd.concat([nautical_sbes]).drop_duplicates().reset_index(drop=True)
all_sbp = pd.concat([nautical_sbp, seabed_sbp]).drop_duplicates().reset_index(drop=True)
all_sss = pd.concat([seabed_sss]).drop_duplicates().reset_index(drop=True)
all = [all_mbes, all_sbes, all_sbp, all_sss]

In [71]:
print('The following instruments are suitable regarding only space requirements: ')
print('\n', 'MBES models')
all_mbes[['Model','tx_length', 'tx_width', 'tx_weight', 'rx_length', 'rx_width', 'rx_weight']]
print('\n', 'SBES models')
all_sbes
print('\n', 'SBP models')
all_sbp
print('\n', 'SSS models')
all_sss


The following instruments are suitable regarding only space requirements: 

 MBES models


,Model,tx_length,tx_width,tx_weight,rx_length,rx_width,rx_weight
0,EM304 0.5x0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,EM304 1x1,0.0,0.0,0.0,0.0,0.0,0.0
2,EM304 2x2,0.0,0.0,0.0,0.0,0.0,0.0
3,EM304 4x4,0.0,0.0,0.0,0.0,0.0,0.0
4,EM712 0.5x1,1940.0,224.0,0.0,970.0,224.0,0.0
5,EM712 1x1,970.0,224.0,0.0,970.0,224.0,0.0
6,EM712 1x2,970.0,224.0,0.0,490.0,224.0,0.0
7,EM712 2x2,490.0,224.0,0.0,490.0,224.0,0.0
8,HydroSweepDS 1x2,5658.0,299.0,0.0,299.0,2829.0,0.0
9,HydroSweepDS 2x2,2829.0,299.0,0.0,299.0,2829.0,0.0



 SBES models


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation
0,EA640,Kongsberg,2021-09-29,bathymetry,1.0,12500.0,NaN,NaN,213.0,438.0,...,12-500 kHz,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN



 SBP models


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation,Unnamed: 38
0,deep-36,Innomar,29.09.2021,NaN,5.0,6000,NaN,150.0,900.0,900.0,...,2-7 kHz,NaN,NaN,yes,no,yes,NaN,NaN,"heave, roll",NaN
1,standard,Innomar,29.09.2021,NaN,0.5,500,NaN,50.0,260.0,520.0,...,4-15 kHz,NaN,NaN,yes,no,yes,NaN,NaN,NaN,NaN
2,Echoes3 3500 T7,iXblue,29.09.2021,NaN,1.0,6000,NaN,150.0,980.0,980.0,...,NaN,NaN,NaN,NaN,no,yes,NaN,NaN,NaN,NaN
3,Echoes 3 500 T3,iXblue,29.09.2021,NaN,1.0,1500,NaN,150.0,680.0,680.0,...,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
4,Echoes 3 500 T1,iXblue,29.09.2021,NaN,1.0,200,NaN,150.0,450.0,450.0,...,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
5,SBP29 3,Kongsberg,29.09.2021,NaN,10.0,11000,NaN,200.0,7450.0,800.0,...,NaN,NaN,NaN,no,yes,yes,NaN,NaN,"roll, pitch, yaw",hydrophone shared with EM122/302
6,SBP29 6,Kongsberg,29.09.2021,NaN,10.0,11000,NaN,200.0,3834.0,800.0,...,NaN,NaN,NaN,no,yes,yes,NaN,NaN,"roll, pitch, yaw",hydrophone shared with EM122/302
7,SBP29 12,Kongsberg,29.09.2021,NaN,10.0,11000,NaN,200.0,1970.0,800.0,...,NaN,NaN,NaN,no,yes,yes,NaN,NaN,"roll, pitch, yaw",hydrophone shared with EM122/302
8,TOPAS PS18,Kongsberg,29.09.2021,NaN,20.0,11000,NaN,200.0,1142.0,1140.0,...,0.5-6.0 kHz,NaN,NaN,yes,yes,yes,NaN,NaN,NaN,SAS processing
9,TOPAS PS40,Kongsberg,29.09.2021,NaN,4.0,2000,NaN,75.0,840.0,534.0,...,1-10 kHz,NaN,NaN,yes,yes,yes,NaN,NaN,NaN,SAS processing



 SSS models


,Model,manufacturer,last_updated,application,min_depth,max_depth,depth_rating,penetration_depth,tx_length,tx_width,...,freq_range,freq_range_sec,equiangular,equidistant,parametric,CW,FM,backscatter_resolution,watercolumn_logging,beam_stabilisation


# Dependencies

In [9]:
#%load_ext watermark

# pyton, ipython, packages and machine characteristics
%watermark -v -m -p pandas,numpy,ipywidgets,watermark

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

pandas    : 1.2.4
numpy     : 1.20.1
ipywidgets: 7.6.3
watermark : 2.2.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
CPU cores   : 4
Architecture: 64bit

